In [55]:
import numpy as np
import pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing import sequence

import re

import gensim

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

import sys

In [81]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Input, Embedding, Activation, Flatten, Dense
from keras.layers import Conv1D, MaxPooling1D, Dropout
from keras.models import Model, Sequential

from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import to_categorical
from sklearn.utils import shuffle

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import Adadelta, Adam, rmsprop
from time import sleep

from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Bidirectional 

from keras.layers.merge import concatenate
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [167]:
Train = pd.read_csv('../train_data_2.csv')[:300]
Validation = pd.read_csv('../validation_data_2.csv')[:100]
Test = pd.read_csv('../test_data_2.csv')[:100]

realTweet = []
for tweet in Train['tweet']:
    exec('a=' + tweet)
    realTweet.append(a)
Train['tweet'] = realTweet

realTrainTweet = []
for tweet in Validation['tweet']:
    exec('a=' + tweet)
    realTrainTweet.append(a)
Validation['tweet'] = realTrainTweet

realTrainTweet = []
for tweet in Test['tweet']:
    exec('a=' + tweet)
    realTrainTweet.append(a)
Test['tweet'] = realTrainTweet

In [168]:
def fix_word_key(string):
    upper_part = (re.findall('_.*',string))[0].upper()
    result = re.sub(r'_.*', upper_part, string)
    #print(f'was {string} -> became {result}')
    return result

def DataTransformForLSTM(Train, Validation, Test):
    maxima = 0
    for ar in Train['tweet'].to_list():
        if len(ar)>maxima: maxima = len(ar)
    for ar in Validation['tweet']:
        if len(ar)>maxima: maxima = len(ar)
    for ar in Test['tweet']:
        if len(ar)>maxima: maxima = len(ar)
    max_len = maxima + 1
    print(f'max_len = {max_len}')

    tok = Tokenizer()
    tok.fit_on_texts(Test["tweet"]) #fit on train

    # exctrax features from train
    sequences_train = tok.texts_to_sequences(Train["tweet"])
    sequences_matrix_train = sequence.pad_sequences(sequences_train, maxlen=max_len)

    # exctrax features from validation
    sequences_val = tok.texts_to_sequences(Validation["tweet"])
    sequences_matrix_val = sequence.pad_sequences(sequences_val, maxlen=max_len)

    # exctrax features from test
    sequences_test = tok.texts_to_sequences(Test["tweet"])
    sequences_matrix_test = sequence.pad_sequences(sequences_test, maxlen=max_len)

    word_index = tok.word_index
    #print('Found %s unique tokens' % len(word_index))
    max_words = len(word_index)

    wi1 = len(word_index)

    # fix tokenizer problem
    for key in word_index.keys():
        fixed_key = fix_word_key(key)
        word_index[fixed_key] = word_index.pop(key)

    #print('Found %s unique tokens' % len(word_index))
    wi2 = len(word_index)

    if wi1 != wi2:
        print()
        error = 'lenght of word_index was changed!'
        print(error.upper())
        raise ValueError
    else:
        print('Everything is OK')
        print(len(word_index))

    # делаем Embedding на основе w2v модели
    model = gensim.models.KeyedVectors.load_word2vec_format("../../Embeddings/model.bin", binary=True)

    nb_words = min(max_words, len(word_index))+1 # проверяем где меньше, в нашем датасете или в токенайзере.
    EMBEDDING_DIM = 300 # размерность векторов в нашей модели w2v

    print(f'number of words = {nb_words}')

    embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
    print('embedding_matrix shape', embedding_matrix.shape)

    counter = 0
    for word, i in word_index.items():
        if i >= nb_words: continue
        if word in model.vocab:
            #print(model[word])
            embedding_matrix[i] = model[word]
    #     if counter > nb_words: break
    #     counter += 1


    print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
    #pd.DataFrame(embedding_matrix)
    
    X_train = sequences_matrix_train
    y_train = to_categorical(Train['class'])
    X_val = sequences_matrix_val
    y_val = to_categorical(Validation['class'])
    X_test = sequences_matrix_test
    y_test = to_categorical(Test['class'])
    
    return X_train, y_train, X_test, y_test, X_val, y_val, embedding_matrix

X_train, y_train, X_test, y_test, X_val, y_val, embedding_layer = DataTransformForLSTM(Train, Validation, Test)

In [170]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape, X_val.shape, y_val.shape, 'k', embedding_layer.shape

((300, 34),
 (300, 2),
 (100, 34),
 (100, 2),
 (100, 34),
 (100, 2),
 'k',
 (859, 300))

In [211]:
space = {
            'pooling_size1': hp.choice('pooling_size1', [3, 5]),
            'pooling_size2': hp.choice('pooling_size2', [3, 5]),
    
            'dropout1': hp.uniform('dropout1', .25,.5),
            'dropout2': hp.uniform('dropout2', .25,.5),
            'dropout3': hp.uniform('dropout3', .25,.5),
    
            'dense_size': hp.choice('dense_size', [32, 64, 128]),
            
            'batch_size' : hp.choice('batch_size', [32, 64]),
            'hidden_activation': hp.choice('hidden_activation', ['relu', 'sigmoid', 'tanh', 'selu']),
            'hidden_activation2': hp.choice('hidden_activation2', ['relu', 'sigmoid', 'tanh', 'selu']),
            'optimizer': hp.choice('optimizer',['adadelta', 'adam', 'rmsprop']),
            'loss': hp.choice('loss', ['binary_crossentropy']), 
            'epochs' :  15,
            'activation': 'relu',
            'patience': 5
        }

In [215]:
def model(params):
    rate_drop_lstm  = 0.15 + np.random.rand() * 0.25
    rate_drop_dense = 0.15 + np.random.rand() * 0.25

    embedding_layer_fr   = Embedding(nb_words,
                                    EMBEDDING_DIM,
                                    weights=[embedding_matrix],
                                    input_length=max_len,
                                    trainable=True) 

    embedding_layer      = Embedding(nb_words,
                                    EMBEDDING_DIM,
                                    weights=[embedding_matrix],
                                    input_length=max_len,
                                    trainable=False) 

    embedding_layer_char = Embedding(nb_words,
                                    EMBEDDING_DIM,
                                    weights=[embedding_matrix],
                                    input_length=max_len,
                                    trainable=True) 

    # CONFIGURATION OF BI-LSTM
    lstm_layer           = Bidirectional(LSTM(64, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm))

    sequence_1_input     = Input(shape=(max_len,), dtype='int32')
    embedded_sequences_1 = embedding_layer_fr(sequence_1_input)
    x1                   = lstm_layer(embedded_sequences_1)

    sequence_2_input     = Input(shape=(max_len,), dtype='int32')
    embedded_sequences_2 = embedding_layer(sequence_2_input)
    y1                   = lstm_layer(embedded_sequences_2)

    # CNN topology
    sequence_3_input     = Input(shape=(max_len,), dtype='int64', name='input_CHARS')
    z1                   = embedding_layer_char(sequence_3_input)


    z1                    = Conv1D(filters=300, kernel_size=3, 
                                   activation=params['hidden_activation'], input_shape=(x.shape))(z1)
    z1                    = Dropout(params['dropout1'])(z1)
    z1                    = MaxPooling1D(pool_size=params['pooling_size1'])(z1)

    z1                    = Conv1D(filters=300, kernel_size=5, 
                                   activation=params['hidden_activation'], input_shape=(x.shape))(z1)
    z1                    = Dropout(params['dropout2'])(z1)
    z1                    = MaxPooling1D(pool_size=params['pooling_size2'])(z1)

    z1 = Flatten()(z1)


    num_of_classes = 2
    dropout_p = 0.5
    fully_connected_layers = [200, 200] 
    for dense_size in fully_connected_layers:
        z1 = Dense(dense_size, activation=params['hidden_activation'])(z1)
        z1 = Dropout(params['dropout3'])(z1)



    merged               = concatenate([x1, y1, z1])
    merged               = Dropout(rate_drop_dense)(merged)
    merged               = BatchNormalization()(merged)
    merged               = Dense(32, activation=params['hidden_activation2'])(merged)
    merged               = Dropout(rate_drop_dense)(merged)
    merged               = BatchNormalization()(merged)
    preds                = Dense(2, activation='softmax')(merged)

    STAMP = 'lstm_%d_%d_%.2f_%.2f'%(10, 10, rate_drop_lstm, \
            rate_drop_dense)
    STAMP = 'LSTMCNN_hyperopt'

    model = Model(inputs=[sequence_1_input, sequence_2_input, sequence_3_input], outputs=preds)
    model.compile(loss='binary_crossentropy', 
                  optimizer=params['optimizer'], 
                  metrics=['acc'])
    #model.summary()
    print(STAMP, end='\n\n')

    epochs = params['epochs']
    batch_size = params['batch_size']
    patience = params['patience']

    # form stucture of input data for cases train, val, test
    train_data = [X_train, X_train, X_train]
    val_data   = [X_val, X_val, X_val]
    test_data  = [X_test, X_test, X_test]

    early_stopping =EarlyStopping(monitor='val_loss', patience=patience)
    bst_model_path = STAMP + '.h5'
    model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)

    history = model.fit(train_data, y_train, 
                     validation_data=(val_data, y_val), 
                     epochs=epochs, batch_size=batch_size, shuffle=True,
                     verbose = 0,
                     callbacks=[early_stopping, model_checkpoint])


    check = np.amax(history.history['val_loss'])
    print(f'val_loss: {check:.6f}')
    sys.stdout.flush() 
    return {'loss': check, 'status': STATUS_OK}

import warnings
warnings.filterwarnings('ignore')

trials = Trials()
best = fmin(model, space, algo=tpe.suggest, max_evals=10, trials=trials)
print('\n THE best: ')

for parameter in best: print((parameter, best[parameter]))

LSTMCNN_hyperopt                                      
val_loss: 0.545985                                    
LSTMCNN_hyperopt                                                                
val_loss: 0.677308                                                              
LSTMCNN_hyperopt                                                                
val_loss: 0.568073                                                              
LSTMCNN_hyperopt                                                                
val_loss: 0.664880                                                              
LSTMCNN_hyperopt                                                                
val_loss: 0.364403                                                              
LSTMCNN_hyperopt                                                                 
val_loss: 0.649480                                                               
LSTMCNN_hyperopt                                                              

In [1]:
print(space_eval(space, best))

NameError: name 'space' is not defined

In [204]:
pred = model.predict(test_data)

from sklearn.metrics import classification_report
pred = [round(predict[0], 0) for predict in pred]
real = [t[0] for t in y_test ]
print(classification_report(real, pred))
pd.DataFrame(confusion_matrix(real, bin_pred), columns=['Pos', 'Neg'])

AttributeError: 'function' object has no attribute 'predict'